In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "5"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch

In [2]:
from src.vlstm_v2.interface import testkernel, copykernel, mmkernelv1, mmkernelv2

INCLUDE: ['/scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include', '/scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/TH', '/scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/THC', '/scratch/maximilian.beck/.conda/envs/xlstmdev1/include']
/iarai/home/maximilian.beck/repos/vlstm_cuda/src

/scratch/maximilian.beck/.conda/envs/xlstmdev1/lib


Using /scratch/maximilian.beck/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /scratch/maximilian.beck/.cache/torch_extensions/py311_cu118/vlstm_v1/build.ninja...
Building extension module vlstm_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /scratch/maximilian.beck/.conda/envs/xlstmdev1/bin/nvcc  -ccbin /scratch/maximilian.beck/.conda/envs/xlstmdev1/bin/x86_64-conda-linux-gnu-cc -DTORCH_EXTENSION_NAME=vlstm_v1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include -isystem /scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/TH -isystem /scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/THC -isystem /scratch/maximilian.beck/.conda/envs/xlstmdev1/include -isystem /scratch/maximilian.beck/.conda/envs/xlstmdev1/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS_

Loading extension module vlstm_v1...


In [3]:
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")

In [4]:
mat_A = torch.arange(4).reshape(2, 2).to(dtype=DTYPE, device=DEVICE)
mat_A

tensor([[0., 1.],
        [2., 3.]], device='cuda:0', dtype=torch.bfloat16)

In [5]:
out = testkernel(mat_A)
out

Test kernel!


tensor([[0., 1.],
        [2., 3.]], device='cuda:0', dtype=torch.bfloat16)

In [6]:
out = copykernel(mat_A.to(dtype=torch.float16))
out 

before kernel dispatch - float16!
rows: 2, cols: 2
blocksxy: 1-1, threads: 32-32
cidx: 0, ridx: 0, val: 0.000000
cidx: 1, ridx: 0, val: 2.000000
cidx: 0, ridx: 1, val: 1.000000
cidx: 1, ridx: 1, val: 3.000000


tensor([[0., 1.],
        [2., 3.]], device='cuda:0', dtype=torch.float16)

In [7]:
warp_size = 32
am = torch.ones((1*1*warp_size, 2*1*warp_size), device=DEVICE, dtype=DTYPE)
bm = 2 * torch.ones((2*1*warp_size, 1*1*warp_size), device=DEVICE, dtype=DTYPE)
cm = am @ bm
cm, cm.shape

(tensor([[128., 128., 128.,  ..., 128., 128., 128.],
         [128., 128., 128.,  ..., 128., 128., 128.],
         [128., 128., 128.,  ..., 128., 128., 128.],
         ...,
         [128., 128., 128.,  ..., 128., 128., 128.],
         [128., 128., 128.,  ..., 128., 128., 128.],
         [128., 128., 128.,  ..., 128., 128., 128.]], device='cuda:0',
        dtype=torch.bfloat16),
 torch.Size([32, 32]))

In [8]:
dt = torch.bfloat16
am = am.to(dtype=dt)
bm = bm.to(dtype=dt)
mmkernelv1(mat_A=am, mat_B=bm)

before kernel dispatch - bfloat16!
m: 32, n: 32, k: 64
blocksxy: 4-4, threads: 8-8
cx: 16, cy: 16
cx: 17, cy: 16
cx: 18, cy: 16
cx: 19, cy: 16
cx: 20, cy: 16
cx: 21, cy: 16
cx: 22, cy: 16
cx: 23, cy: 16
cx: 16, cy: 17
cx: 17, cy: 17
cx: 18, cy: 17
cx: 19, cy: 17
cx: 20, cy: 17
cx: 21, cy: 17
cx: 22, cy: 17
cx: 23, cy: 17
cx: 16, cy: 18
cx: 17, cy: 18
cx: 18, cy: 18
cx: 19, cy: 18
cx: 20, cy: 18
cx: 21, cy: 18
cx: 22, cy: 18
cx: 23, cy: 18
cx: 16, cy: 19
cx: 17, cy: 19
cx: 18, cy: 19
cx: 19, cy: 19
cx: 20, cy: 19
cx: 21, cy: 19
cx: 22, cy: 19
cx: 23, cy: 19
cx: 16, cy: 20
cx: 17, cy: 20
cx: 18, cy: 20
cx: 19, cy: 20
cx: 20, cy: 20
cx: 21, cy: 20
cx: 22, cy: 20
cx: 23, cy: 20
cx: 16, cy: 21
cx: 17, cy: 21
cx: 18, cy: 21
cx: 19, cy: 21
cx: 20, cy: 21
cx: 21, cy: 21
cx: 22, cy: 21
cx: 23, cy: 21
cx: 16, cy: 22
cx: 17, cy: 22
cx: 18, cy: 22
cx: 19, cy: 22
cx: 20, cy: 22
cx: 21, cy: 22
cx: 22, cy: 22
cx: 23, cy: 22
cx: 16, cy: 23
cx: 17, cy: 23
cx: 18, cy: 23
cx: 19, cy: 23
cx: 20, cy: 23
cx

tensor([[128., 128., 128.,  ..., 128., 128., 128.],
        [128., 128., 128.,  ..., 128., 128., 128.],
        [128., 128., 128.,  ..., 128., 128., 128.],
        ...,
        [128., 128., 128.,  ..., 128., 128., 128.],
        [128., 128., 128.,  ..., 128., 128., 128.],
        [128., 128., 128.,  ..., 128., 128., 128.]], device='cuda:0',
       dtype=torch.bfloat16)

In [4]:
S = 8 # sequence length
DH = 4 # hidden size

In [5]:
matA = torch.arange((2*S * DH), device=DEVICE, dtype=DTYPE).reshape((2*S, DH))
matB = torch.ones((DH, S), device=DEVICE, dtype=DTYPE)
matA.shape, matB.shape

(torch.Size([16, 4]), torch.Size([4, 8]))

In [6]:
# pytorch
pt_out = matA @ matB
pt_out, pt_out.shape

(tensor([[  6.,   6.,   6.,   6.,   6.,   6.,   6.,   6.],
         [ 22.,  22.,  22.,  22.,  22.,  22.,  22.,  22.],
         [ 38.,  38.,  38.,  38.,  38.,  38.,  38.,  38.],
         [ 54.,  54.,  54.,  54.,  54.,  54.,  54.,  54.],
         [ 70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.],
         [ 86.,  86.,  86.,  86.,  86.,  86.,  86.,  86.],
         [102., 102., 102., 102., 102., 102., 102., 102.],
         [118., 118., 118., 118., 118., 118., 118., 118.],
         [134., 134., 134., 134., 134., 134., 134., 134.],
         [150., 150., 150., 150., 150., 150., 150., 150.],
         [166., 166., 166., 166., 166., 166., 166., 166.],
         [182., 182., 182., 182., 182., 182., 182., 182.],
         [198., 198., 198., 198., 198., 198., 198., 198.],
         [214., 214., 214., 214., 214., 214., 214., 214.],
         [230., 230., 230., 230., 230., 230., 230., 230.],
         [246., 246., 246., 246., 246., 246., 246., 246.]], device='cuda:0',
        dtype=torch.bfloat16),
 torch.

In [7]:
matA.is_contiguous(), matB.is_contiguous()

(True, True)

In [8]:
cu_out = mmkernelv1(mat_A=matA, mat_B=matB)
cu_out, cu_out.shape

before kernel dispatch - bfloat16!
m: 16, n: 8, k: 4
blocksxy: 1-2, threads: 8-8
In Kernel: m: 16, n: 8, k: 4
In Kernel: gdim.x: 1, gdim.y: 2, bdim.x: 8, bdim.y: 8


(tensor([[  6.,   6.,   6.,   6.,   6.,   6.,   6.,   6.],
         [ 22.,  22.,  22.,  22.,  22.,  22.,  22.,  22.],
         [ 38.,  38.,  38.,  38.,  38.,  38.,  38.,  38.],
         [ 54.,  54.,  54.,  54.,  54.,  54.,  54.,  54.],
         [ 70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.],
         [ 86.,  86.,  86.,  86.,  86.,  86.,  86.,  86.],
         [102., 102., 102., 102., 102., 102., 102., 102.],
         [118., 118., 118., 118., 118., 118., 118., 118.],
         [134., 134., 134., 134., 134., 134., 134., 134.],
         [150., 150., 150., 150., 150., 150., 150., 150.],
         [166., 166., 166., 166., 166., 166., 166., 166.],
         [182., 182., 182., 182., 182., 182., 182., 182.],
         [198., 198., 198., 198., 198., 198., 198., 198.],
         [214., 214., 214., 214., 214., 214., 214., 214.],
         [230., 230., 230., 230., 230., 230., 230., 230.],
         [246., 246., 246., 246., 246., 246., 246., 246.]], device='cuda:0',
        dtype=torch.bfloat16),
 torch.

In [9]:
cu_out = mmkernelv2(mat_A=matA, mat_B=matB)
cu_out, cu_out.shape

before kernel dispatch - bfloat16!
m: 16, n: 8, k: 4
blocksxy: 2-4, threads: 4-4


(tensor([[  6.,   6.,   6.,   6.,   6.,   6.,   6.,   6.],
         [ 22.,  22.,  22.,  22.,  22.,  22.,  22.,  22.],
         [ 38.,  38.,  38.,  38.,  38.,  38.,  38.,  38.],
         [ 54.,  54.,  54.,  54.,  54.,  54.,  54.,  54.],
         [ 70.,  70.,  70.,  70.,  70.,  70.,  70.,  70.],
         [ 86.,  86.,  86.,  86.,  86.,  86.,  86.,  86.],
         [102., 102., 102., 102., 102., 102., 102., 102.],
         [118., 118., 118., 118., 118., 118., 118., 118.],
         [134., 134., 134., 134., 134., 134., 134., 134.],
         [150., 150., 150., 150., 150., 150., 150., 150.],
         [166., 166., 166., 166., 166., 166., 166., 166.],
         [182., 182., 182., 182., 182., 182., 182., 182.],
         [198., 198., 198., 198., 198., 198., 198., 198.],
         [214., 214., 214., 214., 214., 214., 214., 214.],
         [230., 230., 230., 230., 230., 230., 230., 230.],
         [246., 246., 246., 246., 246., 246., 246., 246.]], device='cuda:0',
        dtype=torch.bfloat16),
 torch.

In [15]:
# mat @ mat.T @ mat

### pytorch version

In [16]:
H = 6 # hidden size
S = 5 # seq len
B = 1 # batch size
NH = 2 # num heads
DH = H // NH # dim per head
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")
assert H % NH == 0

In [17]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ds = torch.rand((B, NH, S, S), device=DEVICE, dtype=DTYPE)

max_log_D, _ = torch.max(ds.view(B, NH, -1), dim=-1, keepdim=True)  # (B, NH, 1)
log_D_matrix_stabilized = ds - max_log_D.unsqueeze(-1)  # (B, NH, S, S) = (B, NH, S, S) - (B, NH, 1, 1)
D_matrix = torch.exp(log_D_matrix_stabilized)  # (B, NH, S, S)
mval = torch.exp(-max_log_D.unsqueeze(-1))